In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=1)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, rads_bow_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, rads_bow_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     64   10    9  111.93  509  111.17  0.75  0.01  104.15  470  103.09   
10000    44   20   19  111.38  487  110.34  1.03  0.01  103.89  455  102.59   
15000    48   30   29  111.10  474  109.90  1.20  0.00  103.77  448  102.32   
20000    48   40   39  110.93  466  109.60  1.32  0.00  103.65  442  102.10   
25000    48   50   49  110.79  460  109.37  1.42  0.00  103.56  435  101.87   
30000    52   60   59  110.69  456  109.19  1.50  0.00  103.52  432  101.77   
35000    57   70   69  110.62  452  109.05  1.56  0.02  103.55  428  101.69   
40000    49   80   79  110.56  449  108.93  1.61  0.02  103.47  425  101.55   
45000    53   90   89  110.50  446  108.82  1.66  0.03  103.37  421  101.42   
50000    50  100   99  110.45  444  108.72  1.70  0.03  103.34  419  101.35   
55000    53  110  109  110.40  441  108.64  1.74  0.03  103.26  417  101.26   
60000    50  120  119  110.36  440  108.57  1.77  0.03  103.26  416  101.27   
65000    55  130  129  110.33  438  108.50  1.80  0.03  103.19  414  101.15   
70000    50  140  139  110.30  436  108.45  1.82  0.03  103.23  415  101.18   
75000    54  150  149  110.27  435  108.40  1.84  0.03  103.20  414  101.14   
80000    50  160  159  110.24  434  108.35  1.87  0.02  103.23  415  101.17   
85000    55  170  169  110.22  433  108.31  1.88  0.02  103.18  413  101.08   
90000    52  180  179  110.20  432  108.27  1.90  0.02  103.19  414  101.12   
95000    54  190  189  110.18  431  108.24  1.92  0.02  103.35  417  101.25   
100000   51  200  199  110.16  430  108.20  1.93  0.02  103.26  415  101.18   
105000   55  210  209  110.14  429  108.17  1.95  0.02  103.20  413  101.09   
110000   51  220  219  110.13  429  108.15  1.96  0.02  103.20  414  101.12   
115000   55  230  229  110.11  428  108.12  1.97  0.02  103.16  411  101.04   
120000   51  240  239  110.10  427  108.10  1.98  0.02  103.18  413  101.09   
125000   55  250  249  110.09  427  108.07  2.00  0.02  103.12  409  100.97   
130000   51  260  259  110.08  426  108.05  2.01  0.02  103.11  410  101.00   
135000   71  270  269  110.06  426  108.03  2.02  0.02  103.10  409  100.96   
140000   56  280  279  110.05  425  108.01  2.02  0.02  103.09  409  100.95   
145000   54  290  289  110.04  425  107.99  2.03  0.01  103.08  408  100.91   
150000   49  300  299  110.03  424  107.97  2.04  0.01  103.11  410  100.96   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
205000   56  410  409  109.95  420  107.82  2.12  0.01  103.10  406  100.86   
210000   52  420  419  109.95  420  107.81  2.12  0.01  103.11  408  100.91   
215000   57  430  429  109.94  420  107.80  2.13  0.01  103.05  405  100.81   
220000   51  440  439  109.94  419  107.79  2.13  0.01  103.09  407  100.89   
225000   56  450  449  109.93  419  107.78  2.14  0.01  103.04  405  100.81   
230000   52  460  459  109.93  419  107.77  2.14  0.01  103.07  407  100.86   
235000   56  470  469  109.92  419  107.76  2.15  0.01  103.07  406  100.83   
240000   57  480  479  109.92  418  107.75  2.15  0.01  103.10  407  100.88   
245000   56  490  489  109.91  418  107.74  2.16  0.01  103.06  406  100.83   
250000   55  501    0  109.91  418  107.73  2.16  0.01  103.07  406  100.85   
255000   54  511   10  109.90  418  107.73  2.16  0.01  103.05  405  100.80   
260000   51  521   20  109.90  418  107.72  2.17  0.01  103.08  406  100.84   
265000   54  531   30  109.89  417  107.71  2.17  0.01  103.07  405  100.82   
270000   51  541   40  109.89  417  107.70  2.17  0.01  103.10  407  100.87   
275000   55  551   50  109.89  417  107.70  2.18  0.01  103.07  405  100.80   
280000   50  561   60  109.88  417  107.69  2.18  0.01  103.10  407  100.88   
285000   53  571   70  109.88  417  107.68  2.19  0.01  103.07  405  100.80   
290000   47  581   80  109.88  417  107.68  2.19  0.01  103.10  407  100.

0 R: 1.000 P: 0.410 quality price bought 'm ... time - made buy nice
   1 R: 0.300 P: 0.134 pocket inside small nice inch ipad pockets side perfect strap
     11 R: 0.111 P: 0.111 carry pockets room camera shoulder compartments plenty straps pack books
     13 R: 0.056 P: 0.056 mouse power room cord charger pocket drive usb carry cords
   4 R: 0.103 P: 0.061 sleeve protection air pro protect inside smell neoprene snug inch
     41 R: 0.042 P: 0.042 ; & perfectly perfect hp inside dell love big !
   2 R: 0.074 P: 0.033 mac pro perfectly air book protect protects recommend love hard
     22 R: 0.041 P: 0.041 ! love perfect recommend absolutely ... highly awesome buy loves
   3 R: 0.112 P: 0.068 cover bottom keyboard top hard screen scratches plastic speck easily
     31 R: 0.045 P: 0.045 color cover keyboard love picture blue perfectly pink ordered purple
{0: [1, 4, 3, 5], 1: [11, 13, 12], 4: [42], 3: [32], 5: [51]}
